# **Federal Emergency Management Agency (FEMA)**

## *Exploratory Data Analysis*

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../files/DisasterDeclarationsSummaries.csv')
data.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,lastRefresh,hash,id
0,DR-4710-AL,4710,AL,DR,2023-05-05T00:00:00.000Z,2023,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",False,False,...,NaN,1,77,99077,Lauderdale (County),23046,NaN,2023-05-05T19:01:55.559Z,ec0d7ff19f87d4dd064cb5ac611dac3aaf5ac89a,b699892f-79f2-487c-b0c7-6975cf8f5667
1,DR-4710-AL,4710,AL,DR,2023-05-05T00:00:00.000Z,2023,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",False,False,...,NaN,1,103,99103,Morgan (County),23046,NaN,2023-05-05T19:01:55.559Z,03c5f91c2380a01f3d62ce9ee513b21793214226,b8c4f3e0-590c-434b-8f1e-8bed981d5c8f
2,DR-4710-AL,4710,AL,DR,2023-05-05T00:00:00.000Z,2023,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",False,False,...,NaN,1,37,99037,Coosa (County),23046,NaN,2023-05-05T19:01:55.559Z,a1004faaa6e6af5ebba933a1b577eff91d719291,bc0c1acf-b097-4cea-a9af-a92305eccf0c
3,DR-4710-AL,4710,AL,DR,2023-05-05T00:00:00.000Z,2023,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",False,False,...,NaN,1,111,99111,Randolph (County),23046,NaN,2023-05-05T19:01:55.559Z,59880587546b188eecd68104b3fc044a64ceab5a,7e0ba5c4-c777-47b8-91d1-c2709e8fea1f
4,DR-4710-AL,4710,AL,DR,2023-05-05T00:00:00.000Z,2023,Severe Storm,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",False,False,...,NaN,1,123,99123,Tallapoosa (County),23046,NaN,2023-05-05T19:01:55.559Z,d08c74f54a1864a50a786bf77b0d3797d5fc954f,24f76d7f-c876-4725-a8e4-e4c0ffd8b3aa


In [3]:
data.describe()

,disasterNumber,fyDeclared,fipsStateCode,fipsCountyCode,placeCode,declarationRequestNumber
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,4256.210000,2022.69300,27.573000,73.966000,96972.048000,22464.296000
std,550.097592,0.46148,21.478547,57.061922,11657.793782,455.107109
min,3583.000000,2022.00000,1.000000,0.000000,6362.000000,22069.000000
25%,3587.000000,2022.00000,12.000000,31.000000,99031.000000,22088.000000
50%,4673.000000,2023.00000,21.000000,67.000000,99067.000000,22101.000000
75%,4686.000000,2023.00000,45.000000,105.000000,99105.000000,23014.000000
max,5464.000000,2023.00000,72.000000,491.000000,99491.000000,23046.000000
